## Tarea

Implementar un sistema automático reconocedor de caracteres trazados con el mouse, los caracteres son os dígitos del 0 al 9. Por cada dígito, entrenar un modelo oculto de Markov discreto del tipo izquierda-derecha utilizando varios ejemplos de entrenamiento por cada modelo (Ej 3). El clasificador deberá decidir por el carácter asociado al modelo que tenga la mayor probabilidad de haber producido la secuencia de observaciones (secuencia de pendientes cuantificadas) del carácter que se pretende reconocer. Utilizar los algoritmos que vimos en clase (Algoritmo Forward para clasificar y algoritmo Baum-Welch para entrenar), esto quiere decir, no utilizar una librería de modelos ocultos de Markov (A menos que sea HTK o que no usen modelos discretos sino semi-contínuos de Markov). El programa debe leer los movimientos del mouse y reportar el carácter reconocido en cuanto se termine el trazo de dicho carácter.



In [6]:
# Arreglos de observaciones para entrenar
# 0
dig0 = [10, 9, 9, 8, 7, 6, 5, 5, 3, 1, 1, 1, 16, 16, 14, 13, 13, 12]
dig00 = [9, 8, 8, 6, 5, 4, 2, 1, 16, 15, 14, 13, 13, 12]
dig000 = [9, 8, 7, 6, 5, 5, 4, 4, 3, 2, 1, 1, 1, 14, 12, 12, 12, 11, 11, 10]
# 1
dig1 = [5, 5, 5, 5, 5, 5, 5, 5]
dig01 = [5, 5, 5, 5, 5, 5, 4]
dig001 = [5, 5, 5, 5, 5, 5, 5, 5, 6]
# 2
dig2 = [15, 15, 16, 3, 6, 7, 7, 6, 6, 3, 1, 1, 1]
dig02 = [14, 15, 16, 1, 3, 5, 6, 7, 7, 7, 7, 7, 6, 2, 1, 1, 1, 1]
dig002 = [14, 15, 1, 3, 4, 5, 6, 7, 7, 7, 7, 2, 2, 2]
# 3
dig3 = [1, 1, 1, 5, 7, 8, 8, 1, 1, 3, 5, 7, 8, 9]
dig03 = [1, 1, 1, 1, 8, 8, 8, 8, 2, 1, 2, 3, 6, 7, 8, 9, 9]
dig003 = [1, 1, 2, 5, 7, 8, 9, 1, 1, 1, 2, 4, 6, 7, 8, 9, 9]
# 4
dig4 = [9, 9, 9, 9, 9, 9, 16, 16, 16, 15,
        15, 15, 15, 15, 6, 5, 5, 5, 5, 5, 5, 5]
dig04 = [9, 9, 9, 9, 9, 9, 16, 16, 16, 15, 15, 15, 15, 3, 5, 5, 5, 5, 5]
dig004 = [8, 9, 9, 9, 9, 16, 1, 16, 14, 14, 14, 14, 2, 5, 5, 5, 5, 5]
# 5
dig5 = [9, 9, 9, 9, 9, 8, 4, 5, 1, 1, 1, 2, 3, 3, 4, 5, 7, 8, 9, 10, 10]
dig05 = [9, 9, 9, 8, 5, 4, 1, 1, 1, 3, 3, 4, 5, 7, 8, 9, 9]
dig005 = [9, 9, 9, 8, 5, 5, 1, 1, 1, 2, 2, 4, 5, 6, 8, 9, 10, 11, 12]
# 6
dig6 = [8, 7, 7, 6, 5, 4, 2, 1, 16, 14, 13, 10, 8, 8, 7]
dig06 = [9, 8, 7, 6, 5, 4, 4, 3, 2, 2, 1, 15, 13, 12, 10, 9, 8, 7, 6]
dig006 = [8, 8, 7, 6, 5, 5, 4, 3, 2, 1, 15, 13, 10, 8]
# 7
dig7 = [1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7]
dig07 = [1, 1, 1, 1, 1, 1, 3, 6, 6, 6, 6, 5, 6, 6, 6, 6]
dig007 = [1, 1, 1, 1, 1, 3, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6]
# 8
dig8 = [9, 8, 5, 2, 1, 2, 3, 4, 6, 9, 10, 11, 12, 13, 15, 15, 15, 11]
dig08 = [11, 9, 8, 7, 5, 4, 1, 1, 1, 2, 3, 4, 6, 7, 9, 9, 10, 10, 12, 13, 14, 15, 16, 16, 15, 14]
dig008 = [10, 9, 9, 6, 5, 4, 2, 2, 2, 2, 3, 4, 4, 5, 7, 9,
          9, 10, 11, 12, 13, 14, 15, 16, 16, 16, 14, 13, 11, 10]
# 9
dig9 = [10, 8, 6, 5, 2, 1, 13, 13, 5, 5, 5, 5, 5, 5]
dig09 = [9, 8, 6, 5, 3, 1, 15, 13, 13, 5, 5, 5, 5, 5, 5, 5]
dig009 = [9, 9, 7, 4, 2, 16, 13, 14, 5, 5, 4, 4, 5, 5, 5]


In [13]:
import numpy as np
A = np.random.rand(3, 3)
A[1, 0] = 0
A[2, 0] = 0


def stochastic(matrix):
    return matrix/matrix.sum(axis=1)[:, None]


A = stochastic(A)
A


array([[0.27304449, 0.45640348, 0.27055203],
       [0.        , 0.11300221, 0.88699779],
       [0.        , 0.99090543, 0.00909457]])

### Ventana que captura los trazos y emite las observaciones.

In [18]:
from tkinter import *
import math
import numpy as np
app = Tk()
app.geometry("400x400")


canvas = Canvas(app, bg='white')
canvas.pack(anchor='nw', fill='both', expand=1)

xcords = []
ycords = []


def get_x_and_y(event):
    global lasx, lasy
    lasx, lasy = event.x, event.y


def draw(event):
    global lasx, lasy
    xcords.append(lasx)
    ycords.append(lasy)
    canvas.create_line((lasx, lasy, event.x, event.y),
                       fill='red',
                       width=2)
    lasx, lasy = event.x, event.y


def release(event):
    print("solto boton")
    genObser(xcords, ycords)


def genObser(xlist, ylist):
    """Se limpian los arreglos y se regresa la serie de observaciones"""
    xpurge, ypurge = [], []
    deglist = []
    obs = []
    for i in range(0, len(xlist), 20):
        xpurge.append(xlist[i])
        ypurge.append(ylist[i])

    for i in range(len(xpurge)-1):
        dy = ypurge[i]-ypurge[i+1]
        dx = xpurge[i]-xpurge[i+1]
        myradians = math.atan2(dy, dx)
        mydegrees = math.degrees(myradians)
        deglist.append(mydegrees)
        # print(mydegrees)

    for i in deglist:
        # para 1
        if i >= 168.75 and i <= 180:
            obs.append(1)
        if i <= -168.75:
            obs.append(1)
        # para 2
        if i > -168.75 and i < -146.25:
            obs.append(2)
        # 3
        if i > -146.25 and i < -123.75:
            obs.append(3)
        # 4
        if i > -123.75 and i < -101.25:
            obs.append(4)
        # 5
        if i > -101.25 and i < -78.75:
            obs.append(5)
        # 6
        if i > -78.75 and i < -56.25:
            obs.append(6)
        # 7
        if i > -56.25 and i < -33.75:
            obs.append(7)
        # 8
        if i > -33.75 and i < -11.25:
            obs.append(8)
        # 9
        if i >= 0 and i < 11.25:
            obs.append(9)
        if i < 0 and i > -11.25:
            obs.append(9)
        # 10
        if i >= 11.25 and i < 33.75:
            obs.append(10)
        # 11
        if i >= 33.75 and i < 56.25:
            obs.append(11)
        # 12
        if i >= 56.25 and i < 78.75:
            obs.append(12)
        # 13
        if i >= 78.75 and i < 101.25:
            obs.append(13)
        # 14
        if i >= 101.25 and i < 123.75:
            obs.append(14)
        # 15
        if i >= 123.75 and i < 146.25:
            obs.append(15)
        # 16
        if i >= 146.25 and i < 168.75:
            obs.append(16)
    print(f'Secuencia de observaciones {obs}')


canvas.bind("<Button-1>", get_x_and_y)
canvas.bind("<B1-Motion>", draw)
canvas.bind("<ButtonRelease-1>", release)

app.mainloop()


solto boton
Secuencia de observaciones [8, 7, 6, 5, 5, 3, 2, 16, 14, 13, 12, 9, 7, 6, 5]
